In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.wkt
import json      
import requests
import gzip
import haversine
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
import hdbscan
import random
import seaborn as sns

/Users/michaelleong/opt/anaconda3/envs/skmob/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
################
## read files ##
################

#replace LODES files here

#LODES WAC file - in this case, it was pre-aggregated by block group instead of blocks
dmv_wac_2019 = pd.read_csv('/Users/michaelleong/MIT Documents/WMATA - Future of Work and Climate Goals/dmv_wac_2019.csv')

#Census Block Group Shapefile
cbg_cbsa_2010 = gpd.read_file('/Users/michaelleong/MIT Documents/WMATA - Future of Work and Climate Goals/cbg_cbsa_2010.shp')

#CBSA/CSA Shapefile (for filtering)
us_cbsa_geo_2010 = gpd.read_file('/Users/michaelleong/MIT Documents/WMATA - Future of Work and Climate Goals/cb_2019_us_cbsa_500k/cb_2019_us_cbsa_500k.shp')

In [ ]:
######################################
## scatter random points in polygon ##
######################################

def Random_Points_in_Bounds(polygon, number):   
    minx, miny, maxx, maxy = polygon.bounds
    x = np.random.uniform(minx, maxx, number*4)
    y = np.random.uniform(miny, maxy, number*4)
    gdf_poly = gpd.GeoDataFrame(index=["myPoly"], geometry=[polygon])
    df = pd.DataFrame()
    df['points'] = list(zip(x,y))
    df['points'] = df['points'].apply(shapely.geometry.Point)
    gdf_points = gpd.GeoDataFrame(df, geometry='points')
    Sjoin = gpd.sjoin(gdf_points, gdf_poly, op="within", how='left')
    pnts_in_poly = gdf_points[Sjoin.index_right=='myPoly']
    return pnts_in_poly['points'].tolist()[0:number]

#filter CBGs within CBSA (DC in this case)
cbg_cbsa_2010 = cbg_cbsa_2010.rename(columns={'STATEFP':'GEOID10'})
cbg_cbsa_2010['GEOID10'] = cbg_cbsa_2010['GEOID10'].astype(int)
dmv_cbg_cbsa_2010 = cbg_cbsa_2010[cbg_cbsa_2010['CBSA_NAME']=='Washington-Arlington-Alexandria, DC-VA-MD-WV'].reset_index(drop=True)

#join CBGs with LODES
dmv_cbg_cbsa_2010 = dmv_cbg_cbsa_2010.merge(dmv_wac_2019, how="left", left_on='GEOID10', right_on='work_cbg_geoid')

#calculate employment density (extra variable)
dmv_cbg_cbsa_2010['employment_density'] = dmv_cbg_cbsa_2010['C000']/(dmv_cbg_cbsa_2010['ALAND']/4047)

#randomly scatter 1 point in each CBG geo for every 100 jobs (this can be modified)
dmv_cbg_cbsa_2010['jobs_hundreds'] = round((dmv_cbg_cbsa_2010['C000']/100), 0)
dmv_cbg_cbsa_2010 = dmv_cbg_cbsa_2010[dmv_cbg_cbsa_2010['jobs_hundreds']>0].copy()
dmv_cbg_cbsa_2010['points'] = dmv_cbg_cbsa_2010[['geometry', 'jobs_hundreds']].apply(lambda x: Random_Points_in_Bounds(x[0], int(x[1])), axis=1)
dmv_cbg_cbsa_2010_points = dmv_cbg_cbsa_2010.explode('points')

#initialize format for dbscan
dmv_cbg_cbsa_2010_dbscan = gpd.GeoDataFrame(dmv_cbg_cbsa_2010_points[['GEOID10', 'points','employment_density']].copy(), geometry='points')
dmv_cbg_cbsa_2010_dbscan['point_lon'] = dmv_cbg_cbsa_2010_dbscan['points'].x
dmv_cbg_cbsa_2010_dbscan['point_lat'] = dmv_cbg_cbsa_2010_dbscan['points'].y

In [ ]:
###################
## apply HDBSCAN ##
###################

#minimum cluster size can be parameterized. Some literature suggests this be 10,000 jobs (i.e. min_cluster_size = 100).

X = StandardScaler().fit_transform(dmv_cbg_cbsa_2010_dbscan[['point_lon', 'point_lat']].copy())
clusterer = hdbscan.HDBSCAN()
clusterer = hdbscan.HDBSCAN(min_cluster_size=25)
clusterer.fit(X)

In [ ]:
#find number of labels

np.max(clusterer.labels_)

In [ ]:
#append results to original database

dmv_cbg_cbsa_2010_dbscan['cluster'] = clusterer.labels_
dmv_cbg_cbsa_2010_dbscan.to_csv('dmv_clusters_2500_v1.csv')

#remove non-clustered points
dmv_cbg_cbsa_2010_dbscan_by_cbg = dmv_cbg_cbsa_2010_dbscan[dmv_cbg_cbsa_2010_dbscan["cluster"]!=-1].groupby(['GEOID10','cluster']).count().reset_index()

#create mapping of CBGs to subcenters (drop duplicates such that once one CBG point is captured, that CBG is caputred)
#feel free to modify this such that CBGs are only counted if they hit a certain threshold of points, etc.
dmv_cbg_cbsa_2010_dbscan_by_cbg = dmv_cbg_cbsa_2010_dbscan_by_cbg.sort_values(by=['GEOID10','point_lon'], ascending=False).drop_duplicates(['GEOID10'], keep='first')[['GEOID10','cluster']].reset_index(drop=True)
dmv_cbg_cbsa_2010_dbscan_by_cbg_map = dict(zip(dmv_cbg_cbsa_2010_dbscan_by_cbg['GEOID10'],dmv_cbg_cbsa_2010_dbscan_by_cbg['cluster']))
dmv_cbg_cbsa_2010['cluster'] = dmv_cbg_cbsa_2010['GEOID10'].map(dmv_cbg_cbsa_2010_dbscan_by_cbg_map)
dmv_cbg_cbsa_2010[['GEOID10','cluster','geometry']].to_csv('dmv_subcenters_2500_v1.csv')